In [ ]:
from __future__ import print_function

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import math, os, random, cv2
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation

import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from IPython.display import HTML
# Number of workers for dataloader
workers = 2

# Batch size during training
batch_size = 64

# Spatial size of training images. All images will be resized to this
#   size using a transformer.
image_size = 64

# Number of channels in the training images. For color images this is 3
nc = 3

# Size of z latent vector (i.e. size of generator input)
nz = 100

# Size of feature maps in generator
ngf = 64

# Size of feature maps in discriminator
ndf = 64

# Number of training epochs
num_epochs = 100

# Learning rate for optimizers
lr = 0.0006

# Beta1 hyperparam for Adam optimizers
beta1 = 0.5

# Number of GPUs available. Use 0 for CPU mode.
ngpu = 1

#id of the model to save
idmodel=14

In [ ]:
! pip3 install torchgan

In [ ]:
from torchgan.layers import VirtualBatchNorm,MinibatchDiscrimination1d

# Vizualisation

In [ ]:
#from:https://www.kaggle.com/ihelon/monet-eda
def vis(path, n_images, is_random=True, figsize=(16, 16)):
    plt.figure(figsize=figsize)
    
    w = int(n_images ** .5)
    h = math.ceil(n_images / w)
    
    image_names = os.listdir(path)
    
    for i in range(n_images):
        image_name = image_names[i]
        if is_random:
            image_name = random.choice(image_names)
            
        img = cv2.imread(os.path.join(path, image_name))
        img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        plt.subplot(h, w, i + 1)
        plt.imshow(img)
        plt.title(os.path.basename(os.path.normpath(path)))
        plt.xticks([])
        plt.yticks([])
    
    plt.show()
    
BASE_PATH = '../input/simpsons-faces/cropped'
portrait_paint        = os.path.join(BASE_PATH,"")

vis(portrait_paint, 12, is_random=True)

# Datalaoder

In [ ]:
from distutils.dir_util import copy_tree
copy_tree('../input/simpsons-faces/cropped',"simpsons/images")
#opy_tree('../input/wikiart-gangogh-creating-art-gan/portrait')

In [ ]:
#change root_path to "../input/celebrities-100k/100k" or "portrait" to use 100kceleb or portrait paintings
root_path="simpsons"
#root_path="../input/celebrities-100k/100k"
dataset = dset.ImageFolder(root=root_path,
                           transform=transforms.Compose([
                               transforms.Resize(image_size),
                               transforms.CenterCrop(image_size),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ]))
# Create the dataloader
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                         shuffle=True, num_workers=workers,drop_last=True)

device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

In [ ]:
device

In [ ]:
real_batch = next(iter(dataloader))
plt.figure(figsize=(8,8))
plt.axis("off")
plt.title("Training Images")
plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:64], padding=2, normalize=True).cpu(),(1,2,0)))

#  Weight initialization


Using a custom weight initialization with a $\mathcal{N}(0,0.02)$ distribution of weights weight (as suggested in https://arxiv.org/pdf/1511.06434.pdf - 4. DETAILS OF ADVERSARIAL TRAINING) for convolutionnal layers, a  $\mathcal{N}(1,0.02)$ distribution of weights for BatchNorm Layers and 0 BatchNorm bias.

* **Remark** :
The default initialization of BatchNorm layers is constant to 1 for weights and 0 for bias, but it seems that back then when the tutorial was made, the weights in BatchNorm Layers were initialized with a $\mathcal{U}([0,1])$ distribution. The default initialization of Convolutionnal layers seem to be a modified version of the one suggested by Kaiming He (https://arxiv.org/pdf/1502.01852.pdf)

In [ ]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

# Generator

The generator is made of 5 blocks of `ConvTranspose2d`, with `BatchNorm2d` and `ReLu` layers (except for the last one where the activation layer is `Tanh`)

The input data is a random noise (seen later) of size $1\times 1$ with `nz` channels. 
With the size ($4\times 4$) of the kernel and the implicit (or controled) padding in the first `ConvTranspose2d` layer, then with the stride (an reduced padding) the image is upsampled to the $64\times 64$ or `ngf` size with 3 channels.

For each `nn.ConvTranspose2d` the arguments are respectively nn.ConvTranspose2d(`in_channels`, `out_channels`, `kernel_size`, `stride`, `padding`, `bias`)

The actual padding size is `dilation` $\times ($ `kernel_size` $-1) - $ `padding` with default `dilation=1`. So the implicit default padding size is `kernel_size-1`,thus in our example there is an implicit padding for ` nn.ConvTranspose2d( nz, ngf * 8, 4, 1, 0, bias=False) ` of $(3\times 3)$

For this example, considering we have square images and `int` parameters only (so that they have the same size or effect on width and height), an input of size of size ($X \times X\times$`in_channels`) will have an output of size :

<center> $(O,O,\mathrm{out\_channels})$ with $O=(X-1)\times \mathrm{stride} - 2 \times \mathrm{padding} + \mathrm{dilation} \times (\mathrm{kernel\_size} - 1) +1$ </center>

That is how `nn.ConvTranspose2d( nz, ngf * 8, 4, 1, 0, bias=False)` or `nn.ConvTranspose2d( 100, 1024, 4, 1, 0, bias=False)` leads (using 1024 kernels) a $(1\times 1 \times 100)$ tensor to a tensor of size :

<center> $(4,4,1024)$ with $4=(1-1)\times \mathrm{1} - 2 \times \mathrm{0} + \mathrm{1} \times (\mathrm{4} - 1) +1$ </center>


In [ ]:
class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d( nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # state size. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. (ngf*4) x 8 x 8
            nn.ConvTranspose2d( ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state size. (ngf*2) x 16 x 16
            nn.ConvTranspose2d( ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # state size. (ngf) x 32 x 32
            nn.ConvTranspose2d( ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. (nc) x 64 x 64
        )

    def forward(self, input):
        return self.main(input)

In [ ]:
# Create the generator
netG = Generator(ngpu).to(device)

# Apply the weights_init function to randomly initialize all weights
#  to mean=0, stdev=0.2.
netG.apply(weights_init)

# Print the model
print(netG)

# Discriminator

The discriminator is mostly made symmetrically to the Generator with 5 blocks of 2D-Convolution with`LeakyReLu` (which allows negative outputs, with a $0.2$ slope), except for the final convolution, and `BatchNorm2d`, except for the first and the final convolution. 

It takes images of size $3\times 64 \times 64$ as inputs that are convoluted to $512 \times 4 \times 4$ entries then the last convolution yield a $1 \times 1 \times 1$ output where a `Sigmoid` activation is applied.

The `Conv2d` Layers (except the final one) are symetric to the transposed convolutions of the Generator, they use : `stride=2` and `padding=1` (as opposed to the `padding` in transposed convolutions, here the size of the padding is explicit and a line or column of padding of $0$s, default value, is added to the top, the bottom, the left and the right of the inputs). The stride of size 2 yields a quick reduction of the size of the image.

The last `Conv2d` layer is a $4\times 4$ kernel applied on a $512 \times 4 \times 4$ input, thus it is a simple weighted sum for each channel, then a sum over the channels, plus a bias added at the end.

**EDIT** : 
A `main2` block is added (and the activation is removed from the `main` block). The result of the last convolution is set to a $4096\times 1$ tensor, and used as as feature tensor, when needed for feature matching. Then the feature tensor is fed into the `main2` block where it is used with a `linear` layer and a `sigmoid` Activation.
This feature matching method is mentionned in *Improved Techniques for Training GANs* (Salimans et al., 2016), https://arxiv.org/pdf/1606.03498.pdf (the implementation of the feature matching was made using the Tensorflow framework, so this code is inspired by https://github.com/eli5168/improved_gan_pytorch)

An other improvement suggested in Salimans et al. was implemented in version 25, Minibatch Discrimination. It was manually added to the `forward` function, but it slowed down the process (3 hours for 15 epochs, instead of 15 minutes without). 

**EDIT** : 
Added MinibatchDiscrimination using `torchgan`'s `MinibatchDiscrimination1d` layer

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is (nc) x 64 x 64
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf) x 32 x 32
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*2) x 16 x 16
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*4) x 8 x 8
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*8) x 4 x 4
            #nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False)
            nn.Conv2d(ndf * 8, ndf * 16, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 16),
            # state size. (ndf*16) x 2 x 2
        )
        self.main2 = nn.Sequential(
            nn.Linear(4096+128,1),
            nn.Sigmoid()
        )
        self.minibatchdis=nn.Sequential(
            MinibatchDiscrimination1d(in_features =4096,out_features =128,intermediate_features =16),
        )
    def forward(self, input,matching = False):
        output = self.main(input)
        feature = output.view(-1,4096)
        #output = self.main2(feature)
        
        output = output.view(-1,4096)        
        output=self.minibatchdis(output)
        output=self.main2(output)
        
        if matching == True:
            return feature,output
        else:
            return output #batch_size x 1 x 1 x 1 => batch_size

In [ ]:
netD = Discriminator(ngpu).to(device)

# Apply the weights_init function to randomly initialize all weights
#  to mean=0, stdev=0.2.
netD.apply(weights_init)

# Print the model
print(netD)

# Training

We are using the Binary Cross Entroy Loss function with its default parameters.

The `fixed_noise` is only used at the end of epochs or at each 500 iterationns for comparison purposes.

The optimizers used are `Adam` for both networks, there is nos scheduler for the learning rate (there are few epochs) and the fixed learning rate is lower than the suggested learning rate, the $\beta_1$ parameter is at $0.5$ instead of $0.9$ which created to much instability according to the authors of https://arxiv.org/pdf/1511.06434.pdf. The $\beta_2$ parameter is set at its default value.

In [ ]:
# Initialize BCELoss function
criterion = nn.BCELoss()
criterionG = nn.MSELoss()

# Create batch of latent vectors that we will use to visualize
#  the progression of the generator
fixed_noise = torch.randn(64, nz, 1, 1, device=device)

# Establish convention for real and fake labels during training
#Using one-sided-label smoothing, we choose 0.85 as real_label
real_label = 0.85
fake_label = 0.

# Setup Adam optimizers for both G and D
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))

schedulerD = optim.lr_scheduler.StepLR(optimizerD, step_size=int(num_epochs/4), gamma=0.5)
schedulerG = optim.lr_scheduler.StepLR(optimizerG, step_size=int(num_epochs/4), gamma=0.5)

We are using the Binary Cross Entroy Loss function with its default parameters on the discriminator outputs 3 separate times (for each input of each batch it computes $l=-[y.\mathrm{log}(O) + (1-y).\mathrm{log}(1-O)]$ with $O$ the ouput of the considered network, and then the mean across the batch) :
1. On the discriminator for `errD_real` with the output $O=D(x)$ of the discriminator for $x$ real image and with the label $y=1$ thus $l=-\mathrm{log}(D(x))$
2. On the discriminator for `errD_fake` with the output $O=D(G(z))$ with $G$ the generator, and $z$ a random noise used as input for the generator, and with the label $y=0$ thus $l=-\mathrm{log}(1-D(G(z)))$
3. On the discriminator for `errG` with the output $O=D(G(z))$ and with the label $y=1$ thus $l=-\mathrm{log}(D(G(z)))$

In [ ]:
# Training Loop

# Lists to keep track of progress
img_list = []
G_losses = []
D_losses = []
D_G_z1s=[]
D_G_z2s=[]
D_xs=[]
iters = 0

print("Starting Training Loop...")
# For each epoch
for epoch in range(num_epochs):
    # For each batch in the dataloader
    for i, data in enumerate(dataloader, 0):
        ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################
        
        ## Train with all-real batch
        netD.zero_grad()
        real_cpu = data[0].to(device)
        b_size = real_cpu.size(0)
        label = torch.full((b_size,), real_label, dtype=torch.float, device=device)
        
        # Forward pass real batch through D
        output = netD(real_cpu).view(-1)
        errD_real = criterion(output, label)
        
        # Calculate gradients for D in backward pass
        errD_real.backward()
        
        #Calculate the mean value of D(x) with x real images
        D_x = output.mean().item()

              
        ## Train with all-fake batch
        noise = torch.randn(b_size, nz, 1, 1, device=device)
        fake = netG(noise) #generating the fake images
        label.fill_(fake_label) #replace the content of the previous label variable

        # Forward pass fake batch through D
        output = netD(fake.detach()).view(-1)
        errD_fake = criterion(output, label)
        
        # Calculate the gradients for this batch
        errD_fake.backward()
        
        #Calculate the mean value of D(G(z)) with G(z) generated image
        D_G_z1 = output.mean().item()
        
        # Add the gradients from the all-real and all-fake batches
        errD = errD_real + errD_fake
        
        # Update D
        optimizerD.step()
        
        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ###########################
        netG.zero_grad() #reset G gradient
        label.fill_(real_label)  # fake labels are real for generator cost
        
        ##Feature matching
        feature_real,_=netD(real_cpu,matching=True)
        feature_fake,output=netD(fake,matching=True)
        feature_real=torch.mean(feature_real,0)
        feature_fake=torch.mean(feature_fake,0)
        
        errG=criterionG(feature_fake,feature_real.detach())
        
        # Perform another forward pass of all-fake batch through D
        #output = netD(fake).view(-1)
        
        # Calculate G's loss based on this output
        errG += criterion(output.view(-1), label)
        
        # Calculate gradients for G
        errG.backward()
        
        #Calculate the mean value of D(G(z)) with G(z) generated image
        output = netD(fake).view(-1)        
        D_G_z2 = output.mean().item()

        # Update G
        optimizerG.step()

        # Output training stats
        if i % 1 == 0:
            print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                  % (epoch, num_epochs, i, len(dataloader),
                     errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))

        # Save Losses for plotting later
        G_losses.append(errG.item())
        D_losses.append(errD.item())

        D_G_z1s.append(D_G_z1)
        D_G_z2s.append(D_G_z2)
        D_xs.append(D_x)      
        
        # Check how the generator is doing by saving G's output on fixed_noise
        if (iters % 500 == 0) or ((epoch == num_epochs-1) and (i == len(dataloader)-1)):
            with torch.no_grad():
                fake = netG(fixed_noise).detach().cpu()
            img_list.append(vutils.make_grid(fake, padding=2, normalize=True))

        iters += 1
        
    schedulerG.step()
    schedulerD.step()

# Saving the model

In [ ]:
#os.makedirs("models/netD")
#os.makedirs("models/netGS")
#os.makedirs("models/netDS")

#torch.save(netD.state_dict(), "models/netDS/netD_"+str(idmodel)+".pt")
#torch.save(netG.state_dict(), "models/netGS/netG_"+str(idmodel)+".pt")

# Results

In [ ]:
plt.figure(figsize=(10,5))
plt.title("Generator and Discriminator Loss During Training")
plt.plot(G_losses,label="G")
plt.plot(D_losses,label="D")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(10,5))
plt.title("Disciminator mean values")
plt.plot(D_G_z1s,label="D(G(z)) mean values (1)")
plt.plot(D_G_z2s,label="D(G(z)) mean values (2)")
plt.plot(D_xs,label="D(x) mean values")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
#%%capture
fig = plt.figure(figsize=(8,8))
plt.axis("off")
ims = [[plt.imshow(np.transpose(i,(1,2,0)), animated=True)] for i in img_list]
ani = animation.ArtistAnimation(fig, ims, interval=1000, repeat_delay=1000, blit=True)

HTML(ani.to_jshtml())

In [ ]:
# Grab a batch of real images from the dataloader
real_batch = next(iter(dataloader))

# Plot the real images
plt.figure(figsize=(15,15))
plt.subplot(1,2,1)
plt.axis("off")
plt.title("Real Images")
plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:64], padding=5, normalize=True).cpu(),(1,2,0)))

# Plot the fake images from the last epoch
plt.subplot(1,2,2)
plt.axis("off")
plt.title("Fake Images")
plt.imshow(np.transpose(img_list[-1],(1,2,0)))
plt.show()

In [ ]:
plt.imshow(np.transpose(img_list[-1][:,66:132,66:132],(1,2,0)))

# Remove images from the ouput folder

mydir = "./simpson/images"

filelist = [ f for f in os.listdir(mydir)]
for f in filelist:
    os.remove(os.path.join(mydir, f))